In [7]:
library(tidyverse)
library(Seurat)

source('milo_cellbender.R')

In [8]:
other_clusters = qs::qread('../00_cellbender/_targets/objects/obj_cb_other_01') %>% 
    set_labels_to_lvl1 %>%
    set_batch_to_lane %>%    
    prep_obj_for_milo_cb_v01 %>%
    `[[` %>%
    pull(labels) %>% 
    unique
other_clusters

[1] "Tanycytes"         "Oligodendrocytes"  "OPC"              
[4] "Astrocytes"        "Ependymal_cells"   "Microglia"        
[7] "Pericytes"         "Endothelial_cells" "VLMC"

In [9]:
neuron_clusters = qs::qread('../00_cellbender/_targets/objects/obj_cb_neuron_00') %>% 
    set_labels_to_lvl1 %>%
    set_batch_to_lane %>%    
    prep_obj_for_milo_cb_v01 %>%
    `[[` %>%
    pull(labels) %>% 
    unique
neuron_clusters 

[1] "Sst__Pthlh"          "Nfix__Nfib"          "Agrp"               
 [4] "Sim1"                "Fez1"                "Pou6f2__Prkcq__Whrn"
 [7] "Dlk1"                "Slc17a6__Trhr"       "Esr1"               
[10] "Tmem215__Dach2"      "Sst__Nts"            "Sst__Unc13c"        
[13] "Irx5__Lmx1a"         "Htr3b"               "Hdc"                
[16] "Lepr89"              "Nfib_1"              "Ppp1r17__Tafa4"     
[19] "Lepr67"              "Arx__Nr5a2"          "Kiss1__Tac2"        
[22] "Met__Prlh"           "Met__Pappa2"         "Trh__Lef1"          
[25] "Gpr50__Tac2"         "Tmem215__Greb1"      "Ghrh"               
[28] "unassigned__2__"     "Lepr15"              "Pomc__Ttr"          
[31] "Tbx19__Pirt"         "Pomc__Glipr1"        "Pomc__Anxa2"        
[34] "Npsr1__Epha3"        "Trh__Cxcl12"         "Nfib_2"             
[37] "Nfix__Nfib__Lhx6"    "Reln__Lef1"          "Th__Nfib"           
[40] "Ppp1r17"             "Th__Slc6a3"          "Nfib_3"             
[43] "Ppp1r17__Ltbp1"      "Lpar1_oligo"         "Nfix__Nfib__Hgf"

In [10]:
day_splits = tibble(object = rlang::syms(c("exp_labelled_other", "exp_labelled_neuron")))
days = tibble(day = c("Day5", "Day14", "all"))
day_splits = tidyr::crossing(day_splits, days)
day_splits

object,day
<list>,<chr>
exp_labelled_other,all
exp_labelled_other,Day14
exp_labelled_other,Day5
exp_labelled_neuron,all
exp_labelled_neuron,Day14
exp_labelled_neuron,Day5


In [11]:
other_clusters_tibble = tibble(label = other_clusters) %>% 
tidyr::crossing(day_splits) %>%
filter(str_detect(as.character(object), 'other'))


neuron_clusters_tibble = tibble(label = neuron_clusters) %>% 
tidyr::crossing(day_splits) %>%
filter(str_detect(as.character(object), 'neuron'))

clusters_tibble = rbind(other_clusters_tibble, neuron_clusters_tibble)
clusters_tibble = clusters_tibble %>% 
mutate(cluster = label) %>%
rowwise %>%
mutate(label = case_when(day != "all" ~ paste0(c(day, as.character(label)), collapse='.'),
                          TRUE ~ paste0(label, collapse='.'))) %>%
filter(!str_detect(cluster, 'unassigned'))
clusters_tibble

label,object,day,cluster
<chr>,<list>,<chr>,<chr>
Astrocytes,exp_labelled_other,all,Astrocytes
Day14.Astrocytes,exp_labelled_other,Day14,Astrocytes
Day5.Astrocytes,exp_labelled_other,Day5,Astrocytes
Endothelial_cells,exp_labelled_other,all,Endothelial_cells
Day14.Endothelial_cells,exp_labelled_other,Day14,Endothelial_cells
Day5.Endothelial_cells,exp_labelled_other,Day5,Endothelial_cells
Ependymal_cells,exp_labelled_other,all,Ependymal_cells
Day14.Ependymal_cells,exp_labelled_other,Day14,Ependymal_cells
Day5.Ependymal_cells,exp_labelled_other,Day5,Ependymal_cells


In [12]:
qs::qsave(clusters_tibble, 'clusters_tibble_lvl1.qs')